In [1]:
import torch
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer
from tqdm.auto import tqdm
import plotly.io as pio
import numpy as np
import random
import torch.nn as nn
import torch.nn.functional as F
import wandb
import plotly.express as px
import pandas as pd
import torch.nn.init as init
import pickle
import os
from pathlib import Path
from jaxtyping import Int, Float
from torch import Tensor
import einops
import json
from collections import Counter
from datasets import load_dataset
import requests
import pandas as pd
from ipywidgets import interact, IntSlider
import logging
from process_tiny_stories_data import load_tinystories_validation_prompts, load_tinystories_tokens

pio.renderers.default = "notebook_connected"
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

logging.basicConfig(format='(%(levelname)s) %(asctime)s: %(message)s', level=logging.INFO, datefmt='%I:%M:%S')


import sys
sys.path.append('../')  # Add the parent directory to the system path
import utils.haystack_utils as haystack_utils
from autoencoder import AutoEncoder
from utils.autoencoder_utils import custom_forward, AutoEncoderConfig, evaluate_autoencoder_reconstruction, get_encoder_feature_frequencies, load_encoder
import utils.haystack_utils as haystack_utils

%reload_ext autoreload
%autoreload 2

In [2]:
# Run overview
model_name = "tiny-stories-2L-33M"

In [3]:
model = HookedTransformer.from_pretrained(
        model_name,
        center_unembed=True,
        center_writing_weights=True,
        fold_ln=True,
        device=device,
    )

Using pad_token, but it is not set yet.


Loaded pretrained model tiny-stories-2L-33M into HookedTransformer


In [4]:
l0_encoder, l0_cfg = load_encoder("18_morning_sun", model_name, model)
l1_encoder, l1_cfg = load_encoder("2_upbeat_snowball", model_name, model)
print(l0_cfg)
print(l1_cfg)

AutoEncoderConfig(layer=0, act_name='mlp.hook_post', expansion_factor=4, l1_coeff=0.0001)
AutoEncoderConfig(layer=1, act_name='mlp.hook_post', expansion_factor=4, l1_coeff=0.0001)


In [ ]:
original_loss, encoder_loss, zero_ablation_loss = evaluate_autoencoder_reconstruction(l0_encoder, l0_cfg.encoder_hook_point, prompts[:200], model)